In [3]:
import pandas as pd
import os
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Flakify Results

In [24]:
flakify_flaky_test_predict = pd.read_csv('./results/flakify/flaky_test_predict_results.csv')
print('Total Number of test cases:',flakify_flaky_test_predict.shape[0])
print('Number of test cases that flakify incorrectly predicted:', flakify_flaky_test_predict.loc[flakify_flaky_test_predict['flaky'] != flakify_flaky_test_predict['predict']].shape[0])
flakify_flaky_test_predict.head()

Total Number of test cases: 46221
Number of test cases that flakify incorrectly predicted: 432


,Unnamed: 0,project,class,test_name,final_code,flaky,predict
0,0,oozie,TestCoordSubmitXCommand,testSubmitReservedVars,/**\n * Don't include controls in XML.\n *\n *...,1,1
1,1,oozie,TestAuthFilterAuthOozieClient,testClientWithAnonymous,public void testClientWithAnonymous() throws E...,1,0
2,2,oozie,TestBulkMonitorJPAExecutor,testJavaNoRecords,public void testJavaNoRecords() throws Excepti...,1,1
3,3,oozie,TestLiteWorkflowStoreService,testRetry,public void testRetry() throws Exception {\n ...,1,0
4,4,oozie,TestPurgeXCommand,testPurgeWFWithSubWF3,/**\n * Test : The subworkflow should get purg...,1,1


In [9]:
# How many test cases detected in each project
flakify_flaky_test_predict.groupby(by='project').count()

,Unnamed: 0,class,test_name,final_code,flaky,predict
project,,,,,,
achilles,75,75,75,75,75,75
ambari,13,13,13,13,13,13
assertj-core,12024,12024,12024,12024,12024,12024
commons-exec,124,124,124,124,124,124
dropwizard,1730,1730,1730,1730,1730,1730
hadoop,4178,4178,4178,4178,4178,4178
hbase,529,529,529,529,529,529
hector,20,20,20,20,20,20
httpcore,1413,1413,1413,1413,1413,1413


In [25]:
# Generate Accuracy, Precision, Recall, F1 score for each project.
percentage = "{:.0%}".format
def get_evaluation_scores(tn, fp, fn, tp):
    if(tp == 0):
        accuracy = (tp+tn)/(tn+fp+fn+tp)
        Precision = 0
        Recall = 0
        F1 = 0
    else:
        accuracy = (tp+tn)/(tn+fp+fn+tp)
        Precision = tp/(tp+fp)
        Recall = tp/(tp+fn)
        F1 = 2*((Precision*Recall)/(Precision+Recall))
    return percentage(accuracy), percentage(F1), percentage(Precision), percentage(Recall)

result = pd.DataFrame(columns = ['project_name','Accuracy','F1', 'Precision', 'Recall', 'TN', 'FP', 'FN', 'TP'])
for project in flakify_flaky_test_predict['project'].unique():
    data = flakify_flaky_test_predict.loc[flakify_flaky_test_predict['project']==project]
    target = data['flaky']
    preds = data['predict']
    TN, FP, FN, TP = confusion_matrix(target, preds, labels=[0, 1]).ravel()
    accuracy, F1, Precision, Recall = get_evaluation_scores(TN, FP, FN, TP)
    result = result.append(pd.Series([project,accuracy, F1, Precision, Recall, TN, FP, FN, TP], index=result.columns), ignore_index=True)
# result.to_csv('./results/flakify/flakify_per_project_result.csv',  index=False)
result

,project_name,Accuracy,F1,Precision,Recall,TN,FP,FN,TP
0,oozie,75%,76%,100%,62%,415,2,304,488
1,hadoop,99%,96%,95%,98%,3838,17,7,316
2,jimfs,99%,75%,60%,100%,562,6,0,9
3,hbase,99%,96%,95%,97%,454,4,2,69
4,jackrabbit-oak,100%,50%,35%,88%,11483,13,1,7
5,achilles,100%,100%,100%,100%,8,0,0,67
6,orbit,100%,100%,100%,100%,0,0,0,8
7,okhttp,100%,100%,100%,100%,0,0,0,66
8,togglz,95%,50%,33%,100%,447,22,0,11
9,logback,100%,80%,67%,100%,678,1,0,2
